In [ ]:
from datetime import datetime
import pandas as pd
import warnings
import os

warnings.filterwarnings("ignore")

In [ ]:
####################################### ACTUALIZAR CARTERA CCD #######################################

In [ ]:
# Depurar Nuevo_DACxANALISTA
columnas = ['DEUDOR', 'NOMBRE', 'RUC', 'ANALISTA_ACT', 'TIPO_DAC', 'ESTADO']
no_analistas = ['REGION NORTE', 'REGION SUR', 'WALTER LOPEZ', 'SIN INFORMACION']
no_estados = ['LIQUIDADO', 'PROCESO DE LIQUIDACIÓN']

df_dacx = pd.read_excel('docs/Nuevo_DACxANALISTA.xlsx', sheet_name='Base_NUEVA', usecols=columnas)
df_dacx = df_dacx.rename(columns={'ANALISTA_ACT': 'ANALISTA', 'TIPO_DAC': 'TIPO'})
df_dacx = df_dacx[~df_dacx['ANALISTA'].isin(no_analistas)]
df_dacx = df_dacx[~df_dacx['ESTADO'].isin(no_estados)]
df_dacx['DEUDOR'] = df_dacx['DEUDOR'].astype("Int64").astype(str)
df_dacx['RUC'] = df_dacx['RUC'].astype("Int64").astype(str)
df_dacx.dropna(subset=['DEUDOR'], inplace=True)
df_dacx.drop_duplicates(subset=['DEUDOR'], inplace=True)
df_dacx.reset_index(drop=True, inplace=True)

list_analistas = df_dacx['ANALISTA'].unique() # Lista de analistas

# Actualizar CARTERA_GENERAL_CCD
df_cartera = pd.read_excel('docs/APOYOS_CCD.xlsx', sheet_name='GENERAL', usecols=['DEUDOR', 'APOYO1', 'APOYO2', 'APOYO3'])
df_cartera['DEUDOR'] = df_cartera['DEUDOR'].astype("Int64").astype(str)
df_cartera = pd.merge(df_cartera, df_dacx, on='DEUDOR', how='right')
df_cartera = df_cartera[['DEUDOR', 'NOMBRE', 'ANALISTA', 'APOYO1', 'APOYO2', 'APOYO3', 'ESTADO', 'TIPO']]
df_cartera.dropna(subset=['DEUDOR'], inplace=True)
df_cartera.drop_duplicates(subset=['DEUDOR'], inplace=True)
df_cartera.sort_values(by='DEUDOR', inplace=True, ignore_index=True)
df_cartera.reset_index(drop=True, inplace=True)
df_cartera.to_excel('docs/APOYOS_CCD.xlsx', sheet_name='GENERAL', index=False)

# Crear hojas por analista
with pd.ExcelWriter('docs/APOYOS_CCD.xlsx') as writer:
    df_temp = df_cartera.copy()
    df_cartera.to_excel(writer, sheet_name='GENERAL', index=False)
    for analista in list_analistas:
        df_cartera = df_temp[df_temp['ANALISTA'] == analista]
        df_cartera.sort_values(by='DEUDOR', inplace=True, ignore_index=True)
        df_cartera.reset_index(drop=True, inplace=True)
        print(f'HOJA: {analista} [{df_cartera.shape[0]}]')
        df_cartera.to_excel(writer, sheet_name=analista, index=False)

In [ ]:
####################################### BASE CELULARES vs CARTERA CCD #######################################

In [ ]:
df_celulares = pd.read_excel('docs/Base_Celulares_CCD.xlsx')
df_celulares['DEUDOR'] = df_celulares['DEUDOR'].astype("Int64").astype(str)
df_celulares['CELULAR'] = df_celulares['CELULAR'].astype("Int64").astype(str)
df_celulares = df_celulares[['DEUDOR', 'CELULAR']]
df_celulares['CELULAR'] = df_celulares[df_celulares['CELULAR'].str.len() == 9]['CELULAR']
df_celulares.dropna(subset=['CELULAR'], inplace=True)
df_celulares['CELULAR'] = '51' + df_celulares['CELULAR']
df_celulares.reset_index(drop=True, inplace=True)

df_cartera = pd.read_excel('docs/APOYOS_CCD.xlsx', sheet_name='GENERAL', usecols=['DEUDOR', 'ANALISTA', 'APOYO1', 'APOYO2', 'APOYO3'])
df_cartera['DEUDOR'] = df_cartera['DEUDOR'].astype("Int64").astype(str)
df_cruce = pd.merge(df_celulares, df_cartera, on='DEUDOR', how='left')
df_cruce.dropna(subset=['ANALISTA'], inplace=True)
df_cruce.dropna(subset=['CELULAR'], inplace=True)
df_cruce.reset_index(drop=True, inplace=True)
df_cruce.head(5)

In [ ]:
####################################### CADENA DE TEXTO #######################################

In [ ]:
# (f'{row["CELULAR"]}{texto1}{row["ANALISTA"]}{texto2}{row["FECHA_RETORNO"]}{texto3}{row["APOYO"]}{texto4}')
df_regla = pd.read_excel('docs/REGLA.xlsx', sheet_name='TEXTO')
texto1 = df_regla['TEXTO'][0]
texto2 = df_regla['TEXTO'][1]
texto3 = df_regla['TEXTO'][2]
texto4 = df_regla['TEXTO'][3]
print(f'{texto1}\n{texto2}\n{texto3}\n{texto4}')

In [ ]:
####################################### ANALISTAS DE VACACIONES #######################################

In [ ]:
fecha_hoy = datetime.now().strftime('%d.%m.%Y')

df_vacaciones = pd.read_excel('docs/VACACIONES.xlsx')
df_vacaciones.dropna(inplace=True)
dict_analistas = dict(zip(df_vacaciones['ANALISTA'], zip(df_vacaciones['FECHA_SALIDA'], df_vacaciones['FECHA_RETORNO'])))
print(dict_analistas)
vacaciones = {}
for analista, fechas in dict_analistas.items():
    if fechas[0] <= fecha_hoy <= fechas[1]:
        print(f'{analista} de vacaciones desde {fechas[0]} hasta {fechas[1]}')
        vacaciones.update({analista: fechas[1]})
if vacaciones == {}:
    print('Ningún analista de vacaciones')
vacaciones

In [ ]:
####################################### AGREGAR COLUMNA FECHA_RETORNO Y APOYO #######################################

In [ ]:
analistas = list(vacaciones.keys())
df_cruce = df_cruce[df_cruce['ANALISTA'].isin(analistas)]
df_cruce['FECHA_RETORNO'] = df_cruce['ANALISTA'].map(vacaciones)
df_cruce['APOYO'] = df_cruce[['APOYO1', 'APOYO2', 'APOYO3']].apply(
    lambda x: x[0] if x[0] not in (analistas) 
    else (x[1] if x[1] not in (analistas) else x[2]), axis=1)
df_cruce['FECHA_RETORNO'] = df_cruce['FECHA_RETORNO'].str.replace('.', '/')
df_cruce.reset_index(drop=True, inplace=True)
df_cruce = df_cruce[['CELULAR', 'ANALISTA', 'FECHA_RETORNO','APOYO']]
df_cruce

In [ ]:
####################################### AGREGAR COLUMNA TEXTO #######################################

In [ ]:
# (f'{row["CELULAR"]}{texto1}{row["ANALISTA"]}{texto2}{row["FECHA_RETORNO"]}{texto3}{row["APOYO"]}{texto4}')
df_cruce['TEXTO'] = df_cruce.apply(lambda row: f'{row["CELULAR"]}{texto1}{row["ANALISTA"]}{texto2}{row["FECHA_RETORNO"]}{texto3}{row["APOYO"]}{texto4}', axis=1)
df_cruce

In [ ]:
####################################### EXPORTAR TXT #######################################

In [ ]:
lista_apoyos = df_cruce["TEXTO"].to_list()
with open('SMS_APOYOS.txt', "w") as f:
    for item in lista_apoyos:
        f.write("%s\n" % item)

os.startfile('SMS_APOYOS.txt')